https://openai.com/sitemap.xml

#**1: Install All the Required Packages**

In [ ]:
!pip -q install langchain langchain-community
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install openai
!pip install tiktoken

In [ ]:
!pip install tokenizers
!pip install faiss-cpu
!pip -q install unstructured

In [ ]:
!pip install numpy==1.24.4
!pip install nltk==3.9.1

#**2: Import All the Required Libraries**

In [ ]:
import sys
import os
import torch
import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

#Create an Environment

In [ ]:
os.environ['OPENAI_API_KEY'] = ""

#**3: Pass the URLs and extract the data from these URLs**

In [ ]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [ ]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [ ]:
data

In [ ]:
len(data)

#**4: Split the Text into Chunks**

In [ ]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [ ]:
text_chunks=text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

In [ ]:
text_chunks[0]

In [ ]:
text_chunks[1]

In [ ]:
text_chunks[2]

#**5: Download the OpenAI Embeddings or Hugging Face Embeddings**

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

#**06: Convert the Text Chunks into Embeddings and Create a Knowledge Base**

In [ ]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

#**07: Create a Large Language Model (LLM) Wrapper**

In [ ]:
llm=ChatOpenAI()

In [ ]:
llm

In [ ]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

#**08: Initialize the Retrieval QA with Source Chain**

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

In [ ]:
result=chain({"question": "How good is Vicuna?"}, return_only_outputs=True)

In [ ]:
result['answer']

In [ ]:
wrapped_text = textwrap.fill(result['answer'], width=500)

In [ ]:
wrapped_text

In [ ]:
result=chain({"question": "How does Llama 2 outperforms other models"}, return_only_outputs=True)

In [ ]:
result['answer']

In [ ]:
wrapped_text = textwrap.fill(result['answer'], width=500)

In [ ]:
wrapped_text

In [ ]:
result=chain({"question": "What is is stableLM?"}, return_only_outputs=True)


In [ ]:
result['answer']

In [ ]:
wrapped_text = textwrap.fill(result['answer'], width=500)

In [ ]:
wrapped_text

In [ ]:
result=chain({"question": "Can you please share some details about MPT-7b Model"}, return_only_outputs=True)

In [ ]:
result['answer']

In [ ]:
wrapped_text = textwrap.fill(result['answer'], width=100)

In [ ]:
wrapped_text

In [ ]:
while True:
  query=input(f"Prompt: ")
  if query == 'exit':
    print('Exiting')
    sys.exit()
  if query =='':
    continue
  result=chain({'question':query})
  print(f"Answer: " +result["answer"])
